In [1]:
from sympy.tensor.tensor import TensorIndexType, TensorHead, tensor_indices
from sympy.tensor.toperators import PartialDerivative
from sympy import symbols, Rational, sin, cos
from sympy import Tuple
import sympy as sp

In [2]:
# Define tensor indices
Lorentz = TensorIndexType("Lorentz", dummy_name=r"\lambda")
mu, nu, rho, sigma, alpha, beta = tensor_indices(r"\mu \nu \rho \sigma \alpha \beta", Lorentz)

In [3]:
# Tensors are defined by their name and type of indices
x = TensorHead("x", [Lorentz]) # coordinates
g = TensorHead("g", [Lorentz,Lorentz]) # metric
Gamma = TensorHead(r"\Gamma", [Lorentz,Lorentz,Lorentz]) # Christoffel symbols
Riemann_tensor = TensorHead(r"R", [Lorentz,Lorentz,Lorentz,Lorentz]) # Riemann tensor
Ricci_tensor = TensorHead(r"R", [Lorentz,Lorentz]) # Ricci tensor

In [4]:
Christoffel_expr = Rational(1,2) * g(rho,sigma)* (
     PartialDerivative(g(-sigma,-mu), x(nu))
    +PartialDerivative(g(-sigma,-nu), x(mu))
    -PartialDerivative(g(-mu,-nu), x(sigma)))

# Riemann tensor expression in terms of Gamma
Riemann_expr = (
    PartialDerivative(Gamma(alpha, -mu, -nu), x(beta))
  - PartialDerivative(Gamma(alpha, -mu, -beta), x(nu))
  + Gamma(alpha, -beta, -rho) * Gamma(rho, -mu, -nu)
  - Gamma(alpha, -nu, -rho) * Gamma(rho, -mu, -beta)
)

Ricci_expr = (
    g(-alpha,beta) * Riemann_tensor(alpha, -mu, -beta, -nu)
)

display(sp.Eq(Gamma(rho,-mu,-nu),Christoffel_expr))
display(sp.Eq(Riemann_tensor(alpha,-mu,-beta,-nu),Riemann_expr))
display(sp.Eq(Ricci_tensor(-mu,-nu),Ricci_expr))

Eq(\Gamma(\rho, -\mu, -\nu), (1/2)*g(\rho, \lambda_0)*(PartialDerivative(g(-\lambda_0, -\mu), x(\nu)) + PartialDerivative(g(-\lambda_0, -\nu), x(\mu)) + (-1)*PartialDerivative(g(-\mu, -\nu), x(\lambda_0))))

Eq(R(\alpha, -\mu, -\beta, -\nu), PartialDerivative(\Gamma(\alpha, -\mu, -\nu), x(\beta)) - PartialDerivative(\Gamma(\alpha, -\mu, -\beta), x(\nu)) + \Gamma(\alpha, -\beta, -\lambda_0)*\Gamma(\lambda_0, -\mu, -\nu) - \Gamma(\alpha, -\nu, -\lambda_0)*\Gamma(\lambda_0, -\mu, -\beta))

Eq(R(-\mu, -\nu), g(-\lambda_1, \lambda_0)*R(\lambda_1, -\mu, -\lambda_0, -\nu))

# Substitute in explicit metric and coordinates

In [5]:
# Coordinates
t = sp.symbols("t")
r = sp.symbols("r", positive=True)
theta, phi = sp.symbols("theta phi")
coords_array = [t,r,theta,phi]

# Define the metric
Q = sp.symbols("Q") # gravitational charge parameter 
metric_array = [[-(1-(2*Q)/r),0,0,0],
                 [0,1/(1-(2*Q)/r),0,0],
                 [0,0,r**2,0],
                 [0,0,0,r**2 * sp.sin(theta)**2]]

In [6]:
# Coordinates
D,d = 10,4
xs = sp.symbols(f"x0:{d}", real=True)
ys = sp.symbols(f"y0:{D-d}", real=True)
coords_array = [*xs, *ys]

# Define the metric
r2 = sum(y**2 for y in ys)
r = sp.sqrt(r2)
H = sp.Function('H')(r)
g_MN = sp.zeros(D)
g_MN[0:d,0:d] = H**(-sp.Rational(1,2)) * sp.diag(*([-1] + [1]*(d-1)))
g_MN[d:D,d:D] = H**(sp.Rational(1,2)) * sp.eye(D-d)

In [7]:
running_repl = {
    g(-mu,-nu): g_MN,
    x(mu): coords_array,
    Lorentz: g_MN
}

### Compute Christoffel as symbolic array

In [8]:
christ_array = Christoffel_expr.replace_with_arrays(running_repl)
running_repl[Gamma(mu,-nu,-rho)] = christ_array

### Compute Riemann tensor as symbolic array

In [9]:
Riemann_array = Riemann_expr.replace_with_arrays(running_repl)
running_repl[Riemann_tensor(mu,-nu,-rho,-sigma)] = Riemann_array

### Compute Ricci tensor as symbolic array

In [10]:
ricci_tensor_array = Ricci_expr.replace_with_arrays(running_repl)

# Simplify and Examine individual components

In [11]:
r_sym = sp.symbols("r", positive=True)
H_, H_p, H_pp = sp.symbols("H H' H''", real=True)
HARMONIC_DIMENSIONS = D-d

def harmonic_derivative_subs(expr):
    # First sub in r and r^2
    expr = expr.subs({r: r_sym,
                      r2: r_sym**2})
    expr = sp.simplify(expr)
    expr = sp.simplify(expr.subs({sp.sqrt(r2): r_sym,
                                  r2: r_sym**2}))
    
    # Then sub symbols of H, H', H'' for its literal derivatives
    expr = sp.simplify(expr.subs({sp.Function('H')(r_sym): H_,
                                  sp.Derivative(sp.Function('H')(r_sym), r_sym): H_p,
                                  sp.Derivative(sp.Function('H')(r_sym), (r_sym, 2)): H_pp}))
    
    # now impose harmonic condition to eliminate H''
    expr = expr.subs({H_pp: -(HARMONIC_DIMENSIONS -1)/r_sym * H_p})
    return sp.simplify(expr)

In [12]:
R_44 = sp.simplify(harmonic_derivative_subs(ricci_tensor_array[4,4]))

In [13]:
R_44

H'**2*(y0**2 - y1**2 - y2**2 - y3**2 - y4**2 - y5**2)/(4*H**2*r**2)